## Before training

In [1]:
import json
data2020 = json.loads(open('data/data2020.json').read())
data2021 = json.loads(open('data/data2021.json').read())
data = data2020 + data2021
print(len(data))

from sklearn.model_selection import train_test_split
data_train, data_val_all = train_test_split(data, random_state=123, test_size=10000)
data_val, data_test = train_test_split(data_val_all, random_state=123, test_size=5000)
print(len(data_train),len(data_val),len(data_test))

from collections import Counter
term2count = Counter([x for docu in data_train for x in docu['meshMajor']])

224897
214897 5000 5000


In [2]:
FREQ_CUTOFF = 5 
term_freq = sorted([term for term, count in term2count.items() if count>=FREQ_CUTOFF])
labels_ref = sorted([z for z in set([y for x in data_train for y in x['meshMajor']]) if z in term_freq]) 
print(len(term2count), len(labels_ref))
class_freq = [term2count[x] for x in labels_ref]
train_num = len(data_train)

25238 18211


In [3]:
import numpy as np
print(
    np.quantile(np.array(list(class_freq)),q=0.25),
    np.quantile(np.array(list(class_freq)),q=0.33),
    np.quantile(np.array(list(class_freq)),q=0.5),
    np.quantile(np.array(list(class_freq)),q=0.66),
    np.quantile(np.array(list(class_freq)),q=0.75))

11.0 14.0 25.0 48.0 75.0


In [4]:
import pickle
pickle.dump(data_train, open('data/data_train.gt2020.rand123','wb')) 
pickle.dump(data_val, open('data/data_val.gt2020.rand123','wb')) 
pickle.dump(data_test, open('data/data_test.gt2020.rand123','wb')) 
pickle.dump(labels_ref, open('data/labels_ref.gt2020.rand123','wb')) 
pickle.dump(class_freq, open('data/class_freq.gt2020.rand123','wb')) 
pickle.dump(train_num, open('data/train_num.gt2020.rand123','wb')) 

#### to choose the proper gamma of map_param in dbloss

In [5]:
r_all = []
for docu in data_train:
    docu_p = [1/term2count[x] for x in docu['meshMajor'] if x in labels_ref]
    docu_p_sum = sum(docu_p)
    r_all.extend([p/docu_p_sum for p in docu_p])
    
import numpy as np
print(np.mean(r_all),np.median(r_all))
## thus 0.05 can be a good one intuitively

0.08130408890765475 0.015785482633527442


In [6]:
(np.mean(r_all) + np.median(r_all)) / 2
## thus 0.05 can be a good one intuitively

0.0485447857705911